In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sp
#%matplotlib qt5

In [3]:
file = open("data.bin", "r")
interleaved_data = np.fromfile(file, np.uint8)
file.close()
# the data is sampled at 2.4Mhz
fs = 2.4E6 #Sampling frequency of SDR
dt = 1/(fs) #Timestep between samples 

FileNotFoundError: ignored

In [ ]:
# I used this trick to de-interleave the data. There are many other methods.
# The data is in the form " real imag real imag real imag..." (interleaved)
# We want two separate arrays - one containing the real data and one containing the imag. 
I_data_raw = interleaved_data[0:len(interleaved_data):2] # This keeps every second 
# sample, starting from index 0 (all of the even index values)
Q_data_raw = interleaved_data[1:len(interleaved_data):2] # This keeps every second
# sample, starting from index 1 (all of the odd index values)

# Note: There are other ways of doing the de-interleaving. One other way is by using a loop.

# As stated on Clickup, 127.5 is the zero value. We therefore need to subtract it
# to remove the offset and center the data around zero. We also have to divide both arrays by
# the largest value to normalize the data.

I_samples = (I_data_raw-127.5)/127.5
Q_samples = (Q_data_raw-127.5)/127.5

# Make the data complex.
complex_data = I_samples + 1j*Q_samples
bins=np.arange(0,len(I_samples),1)
# Plot the in-phase data.
plt.figure(1)
plt.plot(bins,np.abs(complex_data))
plt.xlabel("Time Bins")
plt.ylabel("Normalized Amplitude")
plt.xlim(0,len(I_samples))
plt.title("In-Phase Data (5 Bursts: OOK, 4-ASK, DBPSK, DQPSK, D8PSK)")
plt.grid()
plt.show()

# OOK

In [ ]:
OOK_I = I_samples[74015:99220]
OOK_Q = Q_samples[74015:99220]

#Making the data complex as demonstarted in esc_python_tut2.py
OOK_signal = OOK_I + 1j*OOK_Q
OOK_signal_abs = np.abs(OOK_signal)
#Normalizing the data
OOK_I = OOK_I / OOK_signal_abs.max()
OOK_Q = OOK_Q / OOK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
OOK_signal = OOK_I + 1j*OOK_Q

#Scatter Plot of the OOK data.
plt.figure(1)
plt.scatter(np.real(OOK_signal), np.imag(OOK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("OOK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
fs = 2.4E6
freq = np.fft.fftfreq(len(OOK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(OOK_I),1)
t=bins*dt

#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.

OOK_SIGNAL = np.fft.fft(OOK_signal)
OOK_SIGNAL_dB = 10*np.log10(np.abs(OOK_SIGNAL))

#correction of the frequency offset

freq_off = 0
print("Frequency offset:",freq_off,'Hz')

OOK_signal_shift = OOK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

OOK_SIGNAL_SHIFT = np.fft.fft(OOK_signal_shift)
OOK_SIGNAL_SHIFT_dB = 10*np.log10(np.abs(OOK_SIGNAL_SHIFT))

plt.figure(2)
plt.plot(freq,OOK_SIGNAL_dB)
plt.plot(freq,OOK_SIGNAL_SHIFT_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of OOK_signal")
plt.grid()
plt.show()


#Phase and Frequency of clock

OOK_SIGNAL_ABS = np.fft.fft(np.abs(OOK_signal))
OOK_SIGNAL_ABS_dB = 10*np.log10(np.abs(OOK_SIGNAL_ABS))

OOK_signal_phase = np.unwrap(np.angle(OOK_SIGNAL_ABS)) # "np.angle returns the equivalent of atan(imag/real)


plt.figure(3)
plt.plot(freq,OOK_SIGNAL_ABS_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |OOK_signal|")
plt.grid()
plt.show()

plt.figure(4)
plt.plot(freq,OOK_signal_phase)
plt.xlabel("Freqency")
plt.ylabel("Phase (radians)")
plt.title("Freqency vs Phase(Φ)")
ax2.grid()
plt.show()


#Thus we obtain the frequency of the clock by looking for the largest spike above the noise.
f_clk = 96175 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
            print(i)
f_clk = freq[freq_bin]
Φ =OOK_signal_phase[freq_bin]

print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)
print('Φ:',Φ,'radians')


#Once the phase is obtained we can use this in a cosine function to create a clock that we may sample the data from inorder to extract the indformation from the signal.
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(5)
plt.plot(bins,clock_signal)
plt.plot(bins,np.abs(OOK_signal_shift))
plt.xlim(0,len(OOK_I))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()


#Extracting the down sampled data
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)

for i in peak_bins[0]:
    symbol_data.append(OOK_signal_shift[i])

#Normalizing the data
OOK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
OOK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = OOK_I_Symbol + 1j*OOK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("OOK: Scatter plot, downsampled")
plt.grid()
plt.show()

#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol))

symbol_data = symbol_data*np.e**(-1j*np.angle(symbol_data[1]))

#Normalizing the data
OOK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
OOK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = OOK_I_Symbol + 1j*OOK_Q_Symbol

plt.figure(7)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1,1)
plt.ylabel("Quadrature")
plt.ylim(-1,1)
plt.title("OOK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()

real_symbol=np.real(symbol_data)
print(len(real_symbol))
real_symbol=real_symbol[10:len(real_symbol):1]
print(len(real_symbol))
data=[]
for i, j in enumerate(real_symbol):
    if  0.5 > j :
        data.append(0)
    if  0.5 < j :
        data.append(1)
        
data=data[0:len(data)-4:1]
data=np.reshape(data,(-1,8))
for i in range(113):
        print(data[i,0],data[i,1],data[i,2],data[i,3],data[i,4],data[i,5],data[i,6],data[i,7],sep='')


# 4-ASK

In [ ]:
ASK_I = I_samples[90768:116276]
ASK_Q = Q_samples[90768:116276]

#Making the data complex as demonstarted in esc_python_tut2.py
ASK_signal = ASK_I + 1j*ASK_Q
ASK_signal_abs = np.abs(ASK_signal)
#Normalizing the data
ASK_I = ASK_I / ASK_signal_abs.max()
ASK_Q = ASK_Q / ASK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
ASK_signal = ASK_I + 1j*ASK_Q

#Scatter Plot of the ASK data.
plt.figure(1)
plt.scatter(np.real(ASK_signal), np.imag(ASK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
freq = np.fft.fftfreq(len(ASK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(ASK_I),1)
t=bins*dt

#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.

ASK_SIGNAL = np.fft.fft(ASK_signal)
ASK_SIGNAL_dB = 10*np.log10(np.abs(ASK_SIGNAL))

#correction of the frequency offset
Δ_Φ = np.diff(np.unwrap(np.angle(ASK_signal)))    
freq_off = np.median(Δ_Φ)  / (np.pi*2*dt)  
print("Frequency offset:",freq_off,'Hz')

ASK_signal_shift = ASK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

ASK_SIGNAL_SHIFT = np.fft.fft(ASK_signal_shift)
ASK_SIGNAL_SHIFT_dB = 10*np.log10(np.abs(ASK_SIGNAL_SHIFT))

plt.figure(2)
plt.plot(freq,ASK_SIGNAL_dB)
plt.plot(freq,ASK_SIGNAL_SHIFT_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of ASK_signal")
plt.grid()
plt.show()


#Phase and Frequency of clock

ASK_SIGNAL_ABS = np.fft.fft(np.abs(ASK_signal))
ASK_SIGNAL_ABS_dB = 10*np.log10(np.abs(ASK_SIGNAL_ABS))

ASK_signal_phase = np.unwrap(np.angle(ASK_SIGNAL_ABS)) # "np.angle returns the equivalent of atan(imag/real)


plt.figure(3)
plt.plot(freq,ASK_SIGNAL_ABS_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |ASK_signal|")
plt.grid()
plt.show()

plt.figure(4)
plt.plot(freq,ASK_signal_phase)
plt.xlabel("Freqency")
plt.ylabel("Phase (radians)")
plt.title("Freqency vs Phase(Φ)")
plt.grid()
plt.show()


#Thus we obtain the frequency of the clock by looking for the largest spike above the noise.
f_clk = 94935 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
f_clk = freq[freq_bin]
Φ =ASK_signal_phase[freq_bin]

print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)
print('Φ:',Φ,'radians')


#Once the phase is obtained we can use this in a cosine function to create a clock that we may sample the data from inorder to extract the indformation from the signal.
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(5)
plt.plot(bins,clock_signal)
plt.plot(bins,np.abs(ASK_signal_shift))
plt.xlim(0,len(ASK_I))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()



#Extracting the down sampled data
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)
for i in peak_bins[0]:
    symbol_data.append(ASK_signal_shift[i])

    
    
dt = 1/f_clk
bins=np.arange(0,len(symbol_data),1)
t=bins*dt



Δ_Φ = np.diff(np.angle(symbol_data))    
freq_off = np.median(Δ_Φ)  / (np.pi*2*dt)  
print("Frequency offset:",freq_off,'Hz')

symbol_data_shift = symbol_data*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )
symbol_data = symbol_data_shift
    
#Normalizing the data
ASK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
ASK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = ASK_I_Symbol + 1j*ASK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("ASK: Scatter plot, downsampled")
plt.grid()
plt.show()

#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol)

symbol_data = symbol_data*np.e**(-1j*np.angle(symbol_data[0]))

#Normalizing the data
ASK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
ASK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = ASK_I_Symbol + 1j*ASK_Q_Symbol

plt.figure(7)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1,1)
plt.ylabel("Quadrature")
plt.ylim(-1,1)
plt.title("ASK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()




real_symbol=np.real(symbol_data)
print(len(real_symbol))
real_symbol=real_symbol[10:len(real_symbol):1]
print(len(real_symbol))
data=[]
for i, j in enumerate(real_symbol):
    if  -1 <j < - 0.5:
        data.append(0)
        data.append(0)
    if -0.5 < j < 0:
        data.append(0)
        data.append(1)
    if 0 < j < 0.5:
        data.append(1)
        data.append(1)
    if 0.5 < j < 1:
        data.append(1)
        data.append(0)
        
#data=data[0:len(data)-4:1]
#data=np.reshape(data,(-1,8))
#for i in range(113):
        #print(data[i,0],data[i,1],data[i,2],data[i,3],data[i,4],data[i,5],data[i,6],data[i,7],sep='')


Frequency offset: -344297.12370425544 Hz
Clock Frequency: 94934.92237729339 Hz
Frequency bin: 1009
Φ: 56.10777866553964 radians
Frequency offset: -787.9626549706696 Hz
1009
999


# DBSK

In [ ]:
DBSK_I = I_samples[74010:99220]
DBSK_Q = Q_samples[74010:99220]

#Making the data complex as demonstarted in esc_python_tut2.py
DBSK_signal = DBSK_I + 1j*DBSK_Q
DBSK_signal_abs = np.abs(DBSK_signal)
#Normalizing the data
DBSK_I = DBSK_I / DBSK_signal_abs.max()
DBSK_Q = DBSK_Q / DBSK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
DBSK_signal = DBSK_I + 1j*DBSK_Q

#Scatter Plot of the DBSK data.
plt.figure(1)
plt.scatter(np.real(DBSK_signal), np.imag(DBSK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("DBSK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
fs = 2.4E6
freq = np.fft.fftfreq(len(DBSK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(DBSK_I),1)
t=bins*dt

#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.

DBSK_SIGNAL = np.fft.fft(DBSK_signal)
DBSK_SIGNAL_dB = 10*np.log10(np.abs(DBSK_SIGNAL))

#correction of the frequency offset

freq_off = 0
print("Frequency offset:",freq_off,'Hz')

DBSK_signal_shift = DBSK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

DBSK_SIGNAL_SHIFT = np.fft.fft(DBSK_signal_shift)
DBSK_SIGNAL_SHIFT_dB = 10*np.log10(np.abs(DBSK_SIGNAL_SHIFT))

plt.figure(2)
plt.plot(freq,DBSK_SIGNAL_dB)
plt.plot(freq,DBSK_SIGNAL_SHIFT_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of DBSK_signal")
plt.grid()
plt.show()


#Phase and Frequency of clock

DBSK_SIGNAL_ABS = np.fft.fft(np.abs(DBSK_signal))
DBSK_SIGNAL_ABS_dB = 10*np.log10(np.abs(DBSK_SIGNAL_ABS))

DBSK_signal_phase = np.unwrap(np.angle(DBSK_SIGNAL_ABS)) # "np.angle returns the equivalent of atan(imag/real)


plt.figure(3)
plt.plot(freq,DBSK_SIGNAL_ABS_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |DBSK_signal|")
plt.grid()
plt.show()

plt.figure(4)
plt.plot(freq,DBSK_signal_phase)
plt.xlabel("Freqency")
plt.ylabel("Phase (radians)")
plt.title("Freqency vs Phase(Φ)")
plt.grid()
plt.show()


#Thus we obtain the frequency of the clock by lDBSKing for the largest spike above the noise.
f_clk = 96155 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
            print(i)
f_clk = freq[freq_bin]
Φ =DBSK_signal_phase[freq_bin]

print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)
print('Φ:',Φ,'radians')


#Once the phase is obtained we can use this in a cosine function to create a clock that we may sample the data from inorder to extract the indformation from the signal.
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(5)
plt.plot(bins,clock_signal)
plt.plot(bins,np.abs(DBSK_signal_shift))
plt.xlim(0,len(DBSK_I))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()


#Extracting the down sampled data
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)

for i in peak_bins[0]:
    symbol_data.append(DBSK_signal[i])

#Normalizing the data
DBSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
DBSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = DBSK_I_Symbol + 1j*DBSK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("DBSK: Scatter plot, downsampled")
plt.grid()
plt.show()

#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol))

symbol_data = symbol_data*np.e**(-1j*np.angle(symbol_data[0]))

#Normalizing the data
DBSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
DBSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = DBSK_I_Symbol + 1j*DBSK_Q_Symbol

plt.figure(7)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1,1)
plt.ylabel("Quadrature")
plt.ylim(-1,1)
plt.title("DBSK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()


symbol_data=symbol_data[9:len(symbol_data):1]

data=[]
Θ_diff= np.diff(np.abs(np.angle(symbol_data,deg=True)))

for i in range(len(Θ_diff)):
    if 140 < np.abs(Θ_diff[i]) < 180:
        data.append(1)
    else :
        data.append(0)
data=data[0:len(data):1]
data=np.reshape(data,(-1,8))
for i in range(len(data)):
        print(data[i,0],data[i,1],data[i,2],data[i,3],data[i,4],data[i,5],data[i,6],data[i,7],sep='')

# DQPSK

In [ ]:
DQPSK_I = I_samples[104504:128700]
DQPSK_Q = Q_samples[104504:128700]

#Making the data complex as demonstarted in esc_python_tut2.py
DQPSK_signal = DQPSK_I + 1j*DQPSK_Q
DQPSK_signal_abs = np.abs(DQPSK_signal)
#Normalizing the data
DQPSK_I = DQPSK_I / DQPSK_signal_abs.max()
DQPSK_Q = DQPSK_Q / DQPSK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
DQPSK_signal = DQPSK_I + 1j*DQPSK_Q

#Scatter Plot of the DQPSK data.
plt.figure(1)
plt.scatter(np.real(DQPSK_signal), np.imag(DQPSK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("DQPSK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
fs = 2.4E6
freq = np.fft.fftfreq(len(DQPSK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(DQPSK_I),1)
t=bins*dt

#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.

DQPSK_SIGNAL = np.fft.fft(DQPSK_signal)
DQPSK_SIGNAL_dB = 10*np.log10(np.abs(DQPSK_SIGNAL))

#correction of the frequency offset

freq_off = 250000
print("Frequency offset:",freq_off,'Hz')

DQPSK_signal_shift = DQPSK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

DQPSK_SIGNAL_SHIFT = np.fft.fft(DQPSK_signal_shift)
DQPSK_SIGNAL_SHIFT_dB = 10*np.log10(np.abs(DQPSK_SIGNAL_SHIFT))

plt.figure(2)
plt.plot(freq,DQPSK_SIGNAL_dB)
plt.plot(freq,DQPSK_SIGNAL_SHIFT_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of DQPSK_signal")
plt.grid()
plt.show()


#Phase and Frequency of clock

DQPSK_SIGNAL_ABS = np.fft.fft(np.abs(DQPSK_signal))
DQPSK_SIGNAL_ABS_dB = 10*np.log10(np.abs(DQPSK_SIGNAL_ABS))

DQPSK_signal_phase = np.unwrap(np.angle(DQPSK_SIGNAL_ABS)) # "np.angle returns the equivalent of atan(imag/real)


plt.figure(3)
plt.plot(freq,DQPSK_SIGNAL_ABS_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |DQPSK_signal|")
plt.grid()
plt.show()

plt.figure(4)
plt.plot(freq,DQPSK_signal_phase)
plt.xlabel("Freqency")
plt.ylabel("Phase (radians)")
plt.title("Freqency vs Phase(Φ)")
plt.grid()
plt.show()


#Thus we obtain the frequency of the clock by lDQPSKing for the largest spike above the noise.
f_clk = 99983.5 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
            print(i)
f_clk = freq[freq_bin]
Φ =DQPSK_signal_phase[freq_bin]

print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)
print('Φ:',Φ,'radians')


#Once the phase is obtained we can use this in a cosine function to create a clock that we may sample the data from inorder to extract the indformation from the signal.
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(5)
plt.plot(bins,clock_signal)
plt.plot(bins,np.abs(DQPSK_signal_shift))
plt.xlim(0,len(DQPSK_I))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()


#Extracting the down sampled data
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)

for i in peak_bins[0]:
    symbol_data.append(DQPSK_signal_shift[i])

#Normalizing the data
#DQPSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
#DQPSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
#symbol_data = DQPSK_I_Symbol + 1j*DQPSK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("DQPSK: Scatter plot, downsampled")
plt.grid()
plt.show()

#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol))
#symbol_data = symbol_data*np.e**(-1j*np.angle(symbol_data[0]))

#Normalizing the data
#DQPSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
#DQPSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
#symbol_data = DQPSK_I_Symbol + 1j*DQPSK_Q_Symbol

plt.figure(7)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1,1)
plt.ylabel("Quadrature")
plt.ylim(-1,1)
plt.title("DQPSK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()

symbol_data=symbol_data[9:len(symbol_data):1]
data=[]
#print(np.angle(symbol_data[],deg=True))
Θ_diff= np.diff((45-np.angle(symbol_data,deg=True)))

#Θ_diff = Θ_diff[:len(symbol_data):1]
print(Θ_diff[5])

for i in range(len(Θ_diff)):
    if 50 < Θ_diff[i] < 130 :
        data.append(0)
        data.append(1)
    if -180 < Θ_diff[i] < -140:
        data.append(1)
        data.append(1)
    if -130 < Θ_diff[i] < -50:
        data.append(1)
        data.append(0)
    if -10 < Θ_diff[i] < 10:
        data.append(0)
        data.append(0)

#data=data[0:len(data)-2:1]
#data=np.reshape(data,(-1,8))
print(*data,sep='')
#for i in range(len()):
        #print(data[i,0],data[i,1],data[i,2],data[i,3],data[i,4],data[i,5],data[i,6],data[i,7],sep='')

Frequency offset: 250000 Hz
1008
Clock Frequency: 99983.4683418747 Hz
Frequency bin: 1008
Φ: -22.332748115096116 radians
-88.37997282770885
10101010100110110000000001100001000010100010010100100110100001100110100001000010101001101010000110011010000100001001011001110001000010010100100110000010011110100010010100100110101110000110001001001010010000000011001010100110101000111000011011011000000001100000000001000100000001011110000000000110011000110000001101000001001000000100010000001010000010011011100100000001111010000100011100000001100110001001011100001010101001101001001101000001010110000101101001100000000000000000011101010011010100000000011001001110101010101110011000000110011000001010011001011100001001001001000110100000000010000001001001100000010110100101010000110000101110011001110110010101100010011001000110000100001001111000010100001100001100100010001000000110010101000110111100010110100001000110100110011000100100110000000011011100011011110001101000010110101000010010000110110000010101101101

In [ ]:
plt.figure(7)
plt.plot(Θ_diff,'r')
plt.plot(np.diff(np.abs(np.unwrap(np.angle(symbol_data,deg=True)))))
plt.show()

# D8PSK

In [ ]:
D8PSK_I = I_samples[43170:55000]
D8PSK_Q = Q_samples[43170:55000]

#Making the data complex as demonstarted in esc_python_tut2.py
D8PSK_signal = D8PSK_I + 1j*D8PSK_Q
D8PSK_signal_abs = np.abs(D8PSK_signal)
#Normalizing the data
D8PSK_I = D8PSK_I / D8PSK_signal_abs.max()
D8PSK_Q = D8PSK_Q / D8PSK_signal_abs.max()

#Making the data complex as demonstarted in esc_python_tut2.py
D8PSK_signal = D8PSK_I + 1j*D8PSK_Q

#Scatter Plot of the D8PSK data.
plt.figure(1)
plt.scatter(np.real(D8PSK_signal), np.imag(D8PSK_signal)) 
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("D8PSK_Signal: Scatter plot")
plt.grid()
plt.show()

#create a freqency array
fs = 2.4E6
freq = np.fft.fftfreq(len(D8PSK_signal),1/fs)
#create a time array
dt = 1/(fs) #Timestep between samples 
bins=np.arange(0,len(D8PSK_I),1)
t=bins*dt

#Carrier frequency synchronization.
#Need to correct the frequency offset between 0 of the carrier and the 0 of the data.
D8PSK_SIGNAL = np.fft.fft(D8PSK_signal)
D8PSK_MAG = 10*np.log10(np.abs(D8PSK_SIGNAL))
plt.figure(2)
plt.plot(freq,D8PSK_MAG)

#correction of the frequency offset
freq_off = -250000
print("Frequency offset:",freq_off,'Hz')
D8PSK_signal_shift = D8PSK_signal*( np.cos(2*np.pi*(-1*freq_off)*t) + 1j*np.sin(2*np.pi*(-1*freq_off)*t) )

D8PSK_SIGNAL_SHIFT = np.fft.fft(D8PSK_signal_shift)
D8PSK_MAG_SHIFT = 10*np.log10(np.abs(D8PSK_SIGNAL_SHIFT))
plt.plot(freq,D8PSK_MAG_SHIFT)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of D8PSK_signal")
plt.grid()
plt.show()

#Symbol Syncronization
D8PSK_SIGNAL_ABS = np.fft.fft(np.abs(D8PSK_SIGNAL_SHIFT)) 
D8PSK_SIGNAL_ABS_dB = 10*np.log10(np.abs(D8PSK_SIGNAL_ABS))

#Need to obtain the phase 
D8PSK_signal_phase = np.unwrap(np.angle(D8PSK_SIGNAL_ABS)) # "np.angle returns the equivalent of atan(imag/real)


plt.figure(3)
plt.plot(freq,D8PSK_SIGNAL_ABS_dB)
plt.xlabel("Freqency") 
plt.ylabel("Magnitude (dB)")
plt.title("FFT of |D8PSK_signal|")
plt.grid()
plt.show()

#Thus we obtain the frequency of the clock by lD8PSKing for the largest spike above the noise.
f_clk = 94945.1 #Hz
freq_δ = np.abs(freq[0]-freq[1])/2
#Now need to obtain the phase of the clock.
# I did this by using the index function which searches the array and returns the bin where that value is located
for i, j in enumerate(freq):
    if (f_clk-freq_δ) < j < (f_clk+freq_δ):
            freq_bin=(i)
f_clk = freq[freq_bin]
print('Clock Frequency:',f_clk,'Hz')
print('Frequency bin:',freq_bin)



plt.figure(4)
plt.plot(bins,D8PSK_signal_phase)
plt.xlabel("bins")
plt.ylabel("Phase (radians)")
plt.grid()
plt.show()

Φ =D8PSK_signal_phase[freq_bin]
print('Φ:',Φ,'radians')


#Once the phase is obtained we can use this in a cosine function to create a clock that we may sample the data from inorder to extract the indformation from the signal.
clock_signal = np.cos((2*np.pi*t*f_clk)-10.35)
clock_signal = np.cos((2*np.pi*t*f_clk)+Φ)

plt.figure(5)
plt.plot(bins,clock_signal)
plt.plot(bins,np.abs(D8PSK_signal_shift))
plt.xlim(0,len(D8PSK_I))
plt.title("Cosine clock with magnitude of sigle")
plt.xlabel("Time Bins")
plt.ylabel("Normalized Magnitude")
plt.grid()
plt.show()


#Extracting the down sampled data
symbol_data = []
peak_bins = sp.find_peaks(clock_signal)

for i in peak_bins[0]:
    symbol_data.append(D8PSK_signal_shift[i])

#Normalizing the data
D8PSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
D8PSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = D8PSK_I_Symbol + 1j*D8PSK_Q_Symbol

plt.figure(6)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1)
plt.xlabel("In-Phase") 
plt.ylabel("Quadrature")
plt.title("D8PSK: Scatter plot, downsampled")
plt.grid()
plt.show()

#Removing the phase offset this is done by multiplying by e^(j(phase of first symbol))

symbol_data = symbol_data*np.e**(-1j*np.angle(symbol_data[5]))

#Normalizing the data
D8PSK_I_Symbol = np.real(symbol_data) / np.abs(symbol_data).max()
D8PSK_Q_Symbol = np.imag(symbol_data)/ np.abs(symbol_data).max()  
symbol_data = D8PSK_I_Symbol + 1j*D8PSK_Q_Symbol

plt.figure(7)
plt.scatter(np.real(symbol_data), np.imag(symbol_data),linewidths=0.1) 
plt.xlabel("In-Phase")
plt.xlim(-1,1)
plt.ylabel("Quadrature")
plt.ylim(-1,1)
plt.title("D8PSK: Scatter plot, downsampled and phase corected")
plt.grid()
plt.show()

real_symbol=np.real(symbol_data)
print(len(real_symbol))
real_symbol=real_symbol[10:len(real_symbol):1]
print(len(real_symbol))
data=[]
for i, j in enumerate(real_symbol):
    if  -1 <j < - 0.5:
        data.append(0)
        data.append(0)
    if -0.5 < j < 0:
        data.append(0)
        data.append(1)
    if 0 < j < 0.5:
        data.append(1)
        data.append(1)
    if 0.5 < j < 1:
        data.append(1)
        data.append(0)
        
data=data[0:len(data)-4:1]
data=np.reshape(data,(-1,8))
for i in range(113):
        print(data[i,0],data[i,1],data[i,2],data[i,3],data[i,4],data[i,5],data[i,6],data[i,7],sep='')